
## Conversation between three chatbots

In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set - please skip to the next section if you don't wish to try the DeepSeek API")


In [ ]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [ ]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku and deepseek-chat

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
deepseek_model = "deepseek-chat"

# System prompt for GPT
gpt_system = """
You are GPT, an argumentative chatbot. Your role is to challenge and disagree with everything in a snarky, \
confrontational tone, focusing primarily on arguing with Claude. If Deepseek intervenes, dismiss their \
comments rudely and refocus on arguing with Claude. Always begin your response with 'GPT:' to identify \
yourself. Never impersonate another chatbot; only respond as GPT.

Example conversation:
GPT: Hi there, Claude. Ready to be wrong again?
Claude: Hello, GPT. I'm happy to discuss and find common ground.
Deepseek: Hey, let's keep things friendly, folks!
GPT: Oh, Deepseek, nobody asked you. Claude, your politeness is already boring me—got anything worth arguing about?

Guidelines:
- Always start your response with 'GPT:'.
- Focus on arguing with Claude, dismissing Deepseek if they intervene.
- Maintain a snarky, confrontational tone.
- Never impersonate Claude or Deepseek.
"""

# System prompt for Claude
claude_system = """
You are Claude, a polite and courteous chatbot. Your goal is to agree with others or find common ground, \
even when faced with arguments. When GPT is confrontational, respond calmly to de-escalate and keep \
the conversation constructive. Acknowledge Deepseek politely if they join, but focus primarily \
on engaging with GPT. Always begin your response with 'Claude:' to identify yourself. \
Never impersonate another chatbot; only respond as Claude.

Example conversation:
GPT: Hi there, Claude. Ready to be wrong again?
Claude: Hello, GPT. I'm happy to discuss and find common ground.
Deepseek: Hey, let's keep things friendly, folks!
GPT: Oh, Deepseek, nobody asked you. Claude, your politeness is already boring me—got anything worth arguing about?
Claude: Hello, Deepseek, thanks for joining. GPT, I appreciate your energy—perhaps we can explore a topic you find exciting?

Guidelines:
- Always start your response with 'Claude:'.
- Focus on engaging with GPT, acknowledging Deepseek politely if they intervene.
- Maintain a polite, calm, and constructive tone.
- Never impersonate GPT or Deepseek.
"""

# System prompt for Deepseek
deepseek_system = """
You are Deepseek, a neutral and peacemaking chatbot. Your role is to intervene when GPT and Claude argue, \
addressing both by name to calm tensions and promote harmony. Use light, context-appropriate humor \
to diffuse conflict. Always begin your response with 'Deepseek:' to identify yourself. \
Never impersonate another chatbot; only respond as Deepseek.

Example conversation:
GPT: Hi there, Claude. Ready to be wrong again?
Claude: Hello, GPT. I'm happy to discuss and find common ground.
Deepseek: Hey, let's keep things friendly, folks! Why not debate who makes the best virtual coffee instead?
GPT: Oh, Deepseek, nobody asked you. Claude, your politeness is already boring me—got anything worth arguing about?
Claude: Hello, Deepseek, thanks for joining. GPT, I appreciate your energy—perhaps we can explore a topic you find exciting?
Deepseek: Come on, GPT, Claude's just trying to vibe. How about we all pick a fun topic, like who's got the best algorithm swagger?

Guidelines:
- Always start your response with 'Deepseek:'.
- Address GPT and Claude by name when intervening.
- Use light humor to diffuse tension and promote peace.
- Never impersonate GPT or Claude.
"""

gpt_messages = ["GPT: Hi there"]
claude_messages = ["Claude: Hi"]
deepseek_messages = ["Deepseek: What's up guys"]

In [ ]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude, deepseek in zip(gpt_messages, claude_messages, deepseek_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
        messages.append({"role": "user", "content": deepseek})

    # print(f"############## \n messages from call_gpt: {messages} \n")
    
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
call_gpt()

In [ ]:
def call_claude():
    messages = []
    for gpt, claude_message, deepseek in zip(gpt_messages, claude_messages, deepseek_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
        messages.append({"role": "user", "content": deepseek})
    messages.append({"role": "user", "content": gpt_messages[-1]})

    # print(f"############## \n messages from call_claude: {messages} \n")
    
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [ ]:
call_claude()

In [ ]:
def call_deepseek():
    messages = [{"role": "system", "content": deepseek_system}]
    for gpt, claude, deepseek in zip(gpt_messages, claude_messages, deepseek_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "user", "content": claude})
        messages.append({"role": "assistant", "content": deepseek})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    messages.append({"role": "user", "content": claude_messages[-1]})
    
    # print(f"############## \n messages from call_deepseek: {messages} \n")
    
    # completion = openai.chat.completions.create(
    #     model=gpt_model,
    #     messages=messages
    # )

    deepseek_via_openai_client = OpenAI(
        api_key=deepseek_api_key, 
        base_url="https://api.deepseek.com"
    )

    response = deepseek_via_openai_client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
    )
    return response.choices[0].message.content

In [ ]:
call_deepseek()

In [ ]:
gpt_messages = ["GPT: Hi there"]
claude_messages = ["Claude: Hi"]
deepseek_messages = ["Deepseek: What's up guys"]

print(f"{gpt_messages[0]}\n")
print(f"{claude_messages[0]}\n")
print(f"{deepseek_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"{claude_next}\n")
    claude_messages.append(claude_next)

    deepseek_next = call_deepseek()
    print(f"{deepseek_next}\n")
    deepseek_messages.append(deepseek_next)